# 画像分類 (classification)プログラム  

ここでは簡単な画像分類プログラムを実行していきながらOpenVINO APIの使用方法を学びます。  

## 必要なライブラリのインポート  

OpenCV (cv2), Numpyなどのライブラリをインポートします。また、Notebook内に画像を表示させるため、PIL (Python Image Library)とIPython.displayもインポートしています。

In [ ]:
import cv2
import numpy as np

import openvino as ov

from PIL import Image
from IPython.display import display

## モデルの読み込みとコンパイル (最適化)  

`resnet50`のモデルを読み込み、`CPU`向けにコンパイルを行います。結果としてコンパイル済みモデルオブジェクトが返されます。  
デバイス名を`CPU`, `GPU`, `NPU`などに変えることで、推論実行デバイスを変更することが可能です。  
複数のGPUデバイスがシステムに存在する場合、`GPU.0`が内蔵GPU、`GPU.1`以降で外付けGPUを指定可能です。  

#### デバイス依存性
`compile_model()` APIは"デバイス非依存"なOpenVINO IRモデルを、デバイス依存の内部形式にコンパイルを行います。この時にデバイス特化の最適化も行われます。  
GPU向けにコンパイルされたコンパイル済みモデルオブジェクトは特定のGPUアーキテクチャや世代に依存するものとなりますので、基本的に他のデバイスに持っていって利用することはできないと考えてください。(`export_model()`, `import_model()`などのAPIを使って他のデバイスで実行できない)

In [ ]:
compiled_model = ov.compile_model('resnet50.xml', device_name='CPU')

## 推論入力データを用意  

OpenCVを使用して、画像ファイルを読み込み、モデルの入力シェイプ(サイズ)に合わせて変形します。  
また、PILとIPythonの機能を利用してnotebook内に読み込んだ画像を表示させています。

#### 入力データのプリプロセス  

入力データはモデルの入力サイズや仕様に合わせて前処理が必要です。ここでは下記の前処理を行っています。  
|処理|内容|
|---|---|
|色並びの変換|OpenCVは画像データを"BGR"のピクセルオーダーでメモリ中に保持します。多くの推論モデルは"RGB"オーダーの画像データを期待しているので、BGR->RGBへの変換を行います|
|リサイズ|モデル入力(1,3,224,224)に合わせて、224x224のサイズにリサイズしています|
|正規化|多くの推論モデルは入力データ範囲として0.0\~1.0、または-1.0\~+1.0のデータを期待しています。また、入力データの中心を0.0に持っていった方が学習がしやすい(収束しやすい)などのメリットがあるため、入力データの平均値を減算したり、偏差を使ったりしてデータの正規化を行う場合があります。**入力データにどのような加工、正規化を行うかは、モデル学習時にどのようなデータ処理を行ったかに依ります**。つまり、推論時に入力するデータは、学習時と同じ前処理が必要ということです。使用するモデルのスペック、仕様を確認してください。<br>ここではピクセル値を255で割って0.0\~1.0の範囲にしたのち、0.5を減算して数値範囲が-0.5\~+0.5になるようにしています。|
|データ軸の入れ替え|OpenCVの画像データはパックドピクセルデータです(チャネルラスト、`HWC`形式)。今回使用するモデルは`NCHW`のチャネルファーストフォーマットですので、Numpyの`transpose()` APIで軸の順番の入れ替えをします。`HWC`(012) -> `CHW`(201)への変換ですので、(2,0,1)を指定しています。|
|軸の追加|ここまでで、データは224x224, `CHW`形式 (3,244,244)への変換が完了していますが、モデルの入力シェイプは`NCHW`の4次元ベクトル(4階 テンソル)です。N==バッチサイズの軸が足りないのでNumpyの`expand_dims()` APIで１つ軸を足して`CHW` -> `NCHW`への変換をしています。ちなみに、`np.reshape(img, (1,3,224,224))`でも同じことが可能です。|

以上のデータ操作を行うことで、`img`のシェイプは(1,3,224,224)となり、モデルの入力シェイプと一致させることができました。

In [ ]:
img = cv2.imread('beaver.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
display(Image.fromarray(img))
img = img.astype(np.float32) / 255.0 - 0.5
img = np.transpose(img, (2,0,1))
img = np.expand_dims(img, axis=0)    # img = np.reshape(img, (1,3,224,224)) これでもよい (こちらのほうが直感的？)
print(img.shape)

## 推論実行、結果表示  

`infer_new_request()` APIで推論を実行します。  

#### 推論結果の後処理  

多くの場合、推論モデルの出力はそのままでは役に立ちません。一定の処理を行い、必要な情報に変換するポストプロセス処理が必要となります。このポストプロセス処理は**モデルによっては画像解析などを伴う重い処理となる場合があります**。見落としがちですが、ディープラーニングを活用したプログラムを作成する場合、プリプロセス、ポストプロセスにかかる時間も含めて１推論サイクルとなるので、推論デバイスのTOPS値以外にも注意を払う必要があります。  

#### プリプロセス、ポストプロセスの並列実行  

今回のチュートリアルでは簡単にしか触れませんが、OpenVINOでは非同期推論APIも提供しています。非同期推論を活用することで推論処理とプリプロセス、ポストプロセスを同時並列に実行させることが可能となり、全体の推論サイクル高速化が可能となります。


In [ ]:
result = compiled_model.infer_new_request(img)    # 推論実行
result = result[0].ravel()                        # 最初の出力を取り出し、フラット化 (1,1000) -> (1000,)  すこし速度が落ちるが.flatten()を使用してもよい

indices = np.argsort(result)[::-1][:5]            # 出力データを数値の小さい順にソート(argsort())した"インデックス"を取得。[::-1]で逆順に並べ替え、先頭の5個を[:5]で取り出す。
print(indices)

for index in indices:                             # 結果を表示。推定されたクラス番号が表示される
    print(index, result[index])

## 結果の表示 (改善)  

推論結果がクラス番号だけでは意味が分からないので、クラス名のテキストファイル`synset_words.txt`を読み込み、クラス名を表示できるように改良します。

In [ ]:
with open('synset_words.txt') as f:
    imagenet_labels = [ line.rstrip().split(maxsplit=1)[1] for line in f.readlines() ]

for index in indices:
    print(f'class_id {index} : {imagenet_labels[index]}, result[index]')

## まとめ

このチュートリアルではシンプルですが実用的な画像分類モデルを使ったプログラムの実際の流れを見てきました。OpenVINOのAPIとしては非常に単純で難しいところはありませんが、前処理、後処理などモデルを活用するためのにはOpenVINO以外のディープラーニングの基礎知識が必要となります。これは他のフレームワークなどを使用した場合でも基本的に同じことです。  
モデルにどのようなデータを入力すればいいか、出てきたデータをどう解釈すればいいかなどは、モデルごとに異なります。使用するモデルの仕様をモデルカードなどの情報を確認してしっかり理解しないとモデルを利用することはできません。